In [2]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the network data
G = nx.read_graphml('london.graphml')

# Load the flow data
flows_df = pd.read_csv('london_flows.csv')

# Basic network information
print("Network Information:")
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

# Basic flow data information
print("\nFlow Data Information:")
print(flows_df.info())

# Check for missing stations
network_stations = set(G.nodes())
flow_stations = set(flows_df['station_origin'].unique()) | set(flows_df['station_destination'].unique())
missing_stations = flow_stations - network_stations

if missing_stations:
    print(f"\nWarning: The following stations are in the flow data but not in the network: {missing_stations}")
else:
    print("\nAll stations in the flow data are present in the network.")

Network Information:
Number of nodes: 401
Number of edges: 467

Flow Data Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61474 entries, 0 to 61473
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   station_origin       61474 non-null  object 
 1   station_destination  61474 non-null  object 
 2   flows                61474 non-null  int64  
 3   population           61474 non-null  int64  
 4   jobs                 61474 non-null  int64  
 5   distance             61474 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 2.8+ MB
None

All stations in the flow data are present in the network.


In [3]:
# Calculate centrality measures
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)

# Create a DataFrame with centrality measures
centrality_df = pd.DataFrame({
    'Degree': degree_centrality,
    'Betweenness': betweenness_centrality,
    'Closeness': closeness_centrality
})

# Display top 10 stations by each centrality measure
print("\nTop 10 stations by Degree Centrality:")
print(centrality_df.sort_values('Degree', ascending=False).head(10))

print("\nTop 10 stations by Betweenness Centrality:")
print(centrality_df.sort_values('Betweenness', ascending=False).head(10))

print("\nTop 10 stations by Closeness Centrality:")
print(centrality_df.sort_values('Closeness', ascending=False).head(10))


Top 10 stations by Degree Centrality:
                          Degree  Betweenness  Closeness
Stratford                 0.0225     0.297846   0.104384
Bank and Monument         0.0200     0.290489   0.113572
King's Cross St. Pancras  0.0175     0.255307   0.113443
Baker Street              0.0175     0.191568   0.108962
Earl's Court              0.0150     0.125892   0.090416
Oxford Circus             0.0150     0.053844   0.111204
Liverpool Street          0.0150     0.270807   0.110254
Waterloo                  0.0150     0.243921   0.112265
Green Park                0.0150     0.215835   0.114778
Canning Town              0.0150     0.096167   0.091575

Top 10 stations by Betweenness Centrality:
                          Degree  Betweenness  Closeness
Stratford                 0.0225     0.297846   0.104384
Bank and Monument         0.0200     0.290489   0.113572
Liverpool Street          0.0150     0.270807   0.110254
King's Cross St. Pancras  0.0175     0.255307   0.113443
Water

In [4]:
def remove_node(G, node):
    G_copy = G.copy()
    G_copy.remove_node(node)
    return G_copy

def calculate_network_metrics(G):
    if not nx.is_connected(G):
        return float('inf'), 0, len(max(nx.connected_components(G), key=len))
    else:
        aspl = nx.average_shortest_path_length(G)
        efficiency = 1 / aspl
        return aspl, efficiency, G.number_of_nodes()

# Perform node removal for top 10 stations by degree centrality
top_stations = centrality_df.sort_values('Degree', ascending=False).head(10).index

results = []
for station in top_stations:
    G_removed = remove_node(G, station)
    aspl, efficiency, component_size = calculate_network_metrics(G_removed)
    results.append({
        'Station': station,
        'ASPL': aspl,
        'Efficiency': efficiency,
        'Largest Component Size': component_size
    })

results_df = pd.DataFrame(results)
print("\nImpact of removing top 10 stations by degree centrality:")
print(results_df)


Impact of removing top 10 stations by degree centrality:
                    Station       ASPL  Efficiency  Largest Component Size
0                 Stratford        inf    0.000000                     379
1         Bank and Monument  14.130739    0.070768                     400
2  King's Cross St. Pancras  14.250890    0.070171                     400
3              Baker Street  14.384624    0.069519                     400
4              Earl's Court  13.832018    0.072296                     400
5             Oxford Circus  13.614925    0.073449                     400
6          Liverpool Street  14.100338    0.070920                     400
7                  Waterloo  13.960802    0.071629                     400
8                Green Park  13.824536    0.072335                     400
9              Canning Town        inf    0.000000                     387


In [5]:
from scipy.optimize import curve_fit

def gravity_model(X, K, beta):
    Pi, Pj, d = X
    return K * Pi * Pj / (d ** beta)

# Prepare data
X = flows_df[['population', 'jobs', 'distance']].values.T
y = flows_df['flows'].values

# Fit the model
popt, _ = curve_fit(gravity_model, X, y, p0=[1e-5, 1])

K_fit, beta_fit = popt
print(f"\nFitted gravity model parameters: K = {K_fit:.2e}, beta = {beta_fit:.2f}")

# Calculate predicted flows
flows_df['predicted_flows'] = gravity_model(X, K_fit, beta_fit)

# Calculate R-squared
ss_res = np.sum((flows_df['flows'] - flows_df['predicted_flows'])**2)
ss_tot = np.sum((flows_df['flows'] - flows_df['flows'].mean())**2)
r_squared = 1 - (ss_res / ss_tot)

print(f"R-squared: {r_squared:.4f}")


Fitted gravity model parameters: K = 1.00e-05, beta = 1.00
R-squared: -inf


/var/folders/c7/kb9rb69966j3v_4115qt2zxh0000gn/T/ipykernel_22029/1098051352.py:5: RuntimeWarning: divide by zero encountered in divide
  return K * Pi * Pj / (d ** beta)
/Users/junrunchen/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [6]:

flows_data = pd.read_csv('london_flows.csv')
flows_data['intervening_opportunities'] = flows_data['jobs'] / flows_data['distance']
canary_wharf_mask = flows_data['station_destination'] == 'Canary Wharf'

# Define the Intervening Opportunities Model function
def intervening_opportunities_model(population, jobs, distance, opportunities, b, alpha):
    """Calculate commuter flows based on population, jobs, distance, and intervening opportunities."""
    return population * jobs * np.exp(-b * distance - alpha * opportunities)

# best values for b and alpha
b_fit_refit = 0.348
alpha_initial = 0.01

# Calculate base scenario flows
flows_data['base_flows'] = intervening_opportunities_model(
    flows_data['population'], flows_data['jobs'], flows_data['distance'],
    flows_data['intervening_opportunities'], b_fit_refit, alpha_initial
)

# Calculate total commuters for baseline
total_commuters_initial = flows_data['base_flows'].sum()

# Scenario A: 50% job reduction at Canary Wharf
flows_data.loc[canary_wharf_mask, 'jobs'] *= 0.5
flows_data['scenario_a_flows'] = intervening_opportunities_model(
    flows_data['population'], flows_data['jobs'], flows_data['distance'],
    flows_data['intervening_opportunities'], b_fit_refit, alpha_initial
)

# Reset jobs to original for accurate scenario B calculations
flows_data.loc[canary_wharf_mask, 'jobs'] = flows_data.loc[canary_wharf_mask, 'jobs'] / 0.5

# Verify total commuters in Scenario A
total_commuters_scenario_a = flows_data['scenario_a_flows'].sum()

# Adjust flows to conserve total commuters if needed
if total_commuters_initial != total_commuters_scenario_a:
    adjustment_factor = total_commuters_initial / total_commuters_scenario_a
    flows_data['scenario_a_flows'] *= adjustment_factor

# Scenario B: Increase in transport cost by adjusting b by 10% and 20%
for i, increase in enumerate([1.10, 1.20], start=1):
    adjusted_b = b_fit_refit * increase
    flows_data[f'scenario_b{i}_flows'] = intervening_opportunities_model(
        flows_data['population'], flows_data['jobs'], flows_data['distance'],
        flows_data['intervening_opportunities'], adjusted_b, alpha_initial
    )

# Print out total flows for each scenario to verify changes
print("Total Base Flows:", total_commuters_initial)
print("Total Scenario A Flows:", flows_data['scenario_a_flows'].sum())
print("Total Scenario B1 Flows:", flows_data['scenario_b1_flows'].sum())
print("Total Scenario B2 Flows:", flows_data['scenario_b2_flows'].sum())

# Sensitivity Analysis with different alpha values
alpha_values = [0.001, 0.01, 0.1]
results = {}

for alpha in alpha_values:
    flows_data['scenario_flows'] = intervening_opportunities_model(
        flows_data['population'], flows_data['jobs'], flows_data['distance'],
        flows_data['intervening_opportunities'], b_fit_refit, alpha
    )
    flows_data[f'scenario_a_flows_alpha_{alpha}'] = intervening_opportunities_model(
        flows_data['population'], flows_data.loc[canary_wharf_mask, 'jobs'] * 0.5, flows_data['distance'],
        flows_data['intervening_opportunities'], b_fit_refit, alpha
    )
    flows_data[f'scenario_b1_flows_alpha_{alpha}'] = intervening_opportunities_model(
        flows_data['population'], flows_data['jobs'], flows_data['distance'],
        flows_data['intervening_opportunities'], b_fit_refit * 1.10, alpha
    )
    flows_data[f'scenario_b2_flows_alpha_{alpha}'] = intervening_opportunities_model(
        flows_data['population'], flows_data['jobs'], flows_data['distance'],
        flows_data['intervening_opportunities'], b_fit_refit * 1.20, alpha
    )

    # Collect results for analysis
    results[alpha] = {
        'Base': flows_data['scenario_flows'].sum(),
        'Scenario A': flows_data[f'scenario_a_flows_alpha_{alpha}'].sum(),
        'Scenario B1': flows_data[f'scenario_b1_flows_alpha_{alpha}'].sum(),
        'Scenario B2': flows_data[f'scenario_b2_flows_alpha_{alpha}'].sum()
    }

# Output results for each alpha variation
print("Detailed Results by Alpha Variation:", results)


Total Base Flows: 7.48084457877715e-31
Total Scenario A Flows: 7.4808445787771495e-31
Total Scenario B1 Flows: 1.3274844971157016e-34
Total Scenario B2 Flows: 2.35580144512799e-38
Detailed Results by Alpha Variation: {0.001: {'Base': 9.842316840059216e-31, 'Scenario A': 1.3892922807262048e-43, 'Scenario B1': 1.7466075540273084e-34, 'Scenario B2': 3.0996802086779167e-38}, 0.01: {'Base': 7.48084457877715e-31, 'Scenario A': 2.760153869557147e-44, 'Scenario B1': 1.3274844971157016e-34, 'Scenario B2': 2.35580144512799e-38}, 0.1: {'Base': 5.195368955947039e-32, 'Scenario A': 2.651339031287365e-51, 'Scenario B1': 9.199470357202758e-36, 'Scenario B2': 1.630313553049657e-39}}


In [10]:
import networkx as nx
import pandas as pd
import numpy as np
from collections import defaultdict

def calculate_centralities(G, flows_df):
    # 度中心性
    degree_cent = nx.degree_centrality(G)
    
    # 加权介数中心性
    edge_weights = {(row['station_origin'], row['station_destination']): 1/(row['flows'] + 1) 
                    for _, row in flows_df.iterrows()}
    nx.set_edge_attributes(G, edge_weights, 'weight')
    betweenness_cent = nx.betweenness_centrality(G, weight='weight')
    
    # 接近中心性
    closeness_cent = nx.closeness_centrality(G)
    
    # 基于流量的加权度中心性
    weighted_degree = defaultdict(float)
    for _, row in flows_df.iterrows():
        weighted_degree[row['station_origin']] += row['flows']
        weighted_degree[row['station_destination']] += row['flows']
    max_flow = max(weighted_degree.values())
    weighted_degree = {k: v/max_flow for k, v in weighted_degree.items()}
    
    return degree_cent, betweenness_cent, closeness_cent, weighted_degree

def assess_removal_impact(G, centrality, flows_df, top_n=10):
    sorted_nodes = sorted(centrality.items(), key=lambda x: x[1], reverse=True)
    impacts = []
    G_copy = G.copy()
    total_flow = flows_df['flows'].sum()
    
    for node, _ in sorted_nodes[:top_n]:
        if node not in G_copy:
            continue
        
        efficiency_before = nx.global_efficiency(G_copy)
        largest_cc_before = len(max(nx.connected_components(G_copy), key=len))
        
        # 计算受影响的流量
        affected_flows = flows_df[(flows_df['station_origin'] == node) | 
                                  (flows_df['station_destination'] == node)]
        flow_impact = affected_flows['flows'].sum() / total_flow
        
        G_copy.remove_node(node)
        
        if nx.is_connected(G_copy):
            efficiency_after = nx.global_efficiency(G_copy)
        else:
            efficiency_after = 0
        largest_cc_after = len(max(nx.connected_components(G_copy), key=len))
        
        impacts.append({
            'node': node,
            'efficiency_change': efficiency_before - efficiency_after,
            'cc_size_change': largest_cc_before - largest_cc_after,
            'flow_impact': flow_impact
        })
    
    return impacts

# 执行分析
degree_cent, betweenness_cent, closeness_cent, weighted_degree = calculate_centralities(G, flows_df)

# 评估不同中心性指标的移除影响
degree_impact = assess_removal_impact(G, degree_cent, flows_df)
betweenness_impact = assess_removal_impact(G, betweenness_cent, flows_df)
closeness_impact = assess_removal_impact(G, closeness_cent, flows_df)
weighted_impact = assess_removal_impact(G, weighted_degree, flows_df)

# 打印结果
for impact_type, impacts in [("Degree", degree_impact), 
                             ("Betweenness", betweenness_impact),
                             ("Closeness", closeness_impact),
                             ("Weighted Degree", weighted_impact)]:
    print(f"\nImpact based on {impact_type} centrality:")
    for impact in impacts[:5]:
        print(f"Node: {impact['node']}")
        print(f"  Efficiency change: {impact['efficiency_change']:.4f}")
        print(f"  Largest component size change: {impact['cc_size_change']}")
        print(f"  Flow impact: {impact['flow_impact']:.4f}")

# 分析哪种中心性指标更好
total_efficiency_change = {
    "Degree": sum(impact['efficiency_change'] for impact in degree_impact),
    "Betweenness": sum(impact['efficiency_change'] for impact in betweenness_impact),
    "Closeness": sum(impact['efficiency_change'] for impact in closeness_impact),
    "Weighted Degree": sum(impact['efficiency_change'] for impact in weighted_impact)
}

best_centrality = max(total_efficiency_change, key=total_efficiency_change.get)
print(f"\nBest centrality measure for identifying critical stations: {best_centrality}")
print("Total efficiency changes:")
for centrality, change in total_efficiency_change.items():
    print(f"  {centrality}: {change:.4f}")


Impact based on Degree centrality:
Node: Stratford
  Efficiency change: 0.1013
  Largest component size change: 22
  Flow impact: 0.0747
Node: Bank and Monument
  Efficiency change: 0.0889
  Largest component size change: 1
  Flow impact: 0.0700
Node: Baker Street
  Efficiency change: 0.0859
  Largest component size change: 1
  Flow impact: 0.0145
Node: King's Cross St. Pancras
  Efficiency change: 0.0820
  Largest component size change: 3
  Flow impact: 0.0400
Node: West Ham
  Efficiency change: 0.0757
  Largest component size change: 3
  Flow impact: 0.0093

Impact based on Betweenness centrality:
Node: Bank and Monument
  Efficiency change: 0.0045
  Largest component size change: 1
  Flow impact: 0.0700
Node: Stratford
  Efficiency change: 0.0967
  Largest component size change: 22
  Flow impact: 0.0747
Node: King's Cross St. Pancras
  Efficiency change: 0.0859
  Largest component size change: 1
  Flow impact: 0.0400
Node: Liverpool Street
  Efficiency change: 0.0803
  Largest comp

In [15]:
import networkx as nx
import pandas as pd
import numpy as np
from collections import defaultdict

def calculate_weighted_centralities(G, flows_df):
    # 加权度中心性
    weighted_degree = defaultdict(float)
    for _, row in flows_df.iterrows():
        if row['station_origin'] in G and row['station_destination'] in G:
            weighted_degree[row['station_origin']] += row['flows']
            weighted_degree[row['station_destination']] += row['flows']
    
    max_flow = max(weighted_degree.values()) if weighted_degree else 1
    weighted_degree = {k: v/max_flow for k, v in weighted_degree.items()}
    
    # 加权介数中心性
    edge_weights = {(row['station_origin'], row['station_destination']): 1/(row['flows'] + 1) 
                    for _, row in flows_df.iterrows() 
                    if row['station_origin'] in G and row['station_destination'] in G}
    nx.set_edge_attributes(G, edge_weights, 'weight')
    weighted_betweenness = nx.betweenness_centrality(G, weight='weight')
    
    # 加权接近中心性
    weighted_closeness = nx.closeness_centrality(G, distance='weight')
    
    return weighted_degree, weighted_betweenness, weighted_closeness

def weighted_global_efficiency(G, flows_df):
    total_efficiency = 0
    total_possible_paths = 0
    
    for _, row in flows_df.iterrows():
        if row['station_origin'] in G and row['station_destination'] in G and row['station_origin'] != row['station_destination']:
            total_possible_paths += 1
            if nx.has_path(G, row['station_origin'], row['station_destination']):
                try:
                    path_length = nx.shortest_path_length(G, row['station_origin'], row['station_destination'], weight='weight')
                    if path_length > 0:
                        total_efficiency += row['flows'] / path_length
                    else:
                        total_efficiency += row['flows']  # 如果路径长度为0，我们假设效率最大
                except nx.NetworkXNoPath:
                    pass  # 如果没有路径，我们不增加效率
    
    return total_efficiency / total_possible_paths if total_possible_paths > 0 else 0

def flow_disruption(G, node, flows_df):
    affected_flows = flows_df[(flows_df['station_origin'] == node) | (flows_df['station_destination'] == node)]
    return affected_flows['flows'].sum() / flows_df['flows'].sum()

def assess_weighted_removal_impact(G, centrality, flows_df, top_n=3):
    sorted_nodes = sorted(centrality.items(), key=lambda x: x[1], reverse=True)
    impacts = []
    
    efficiency_before = weighted_global_efficiency(G, flows_df)
    
    for node, _ in sorted_nodes[:top_n]:
        if node not in G:
            continue
        flow_disruption_value = flow_disruption(G, node, flows_df)
        
        G_copy = G.copy()
        G_copy.remove_node(node)
        
        efficiency_after = weighted_global_efficiency(G_copy, flows_df)
        
        impacts.append({
            'node': node,
            'efficiency_change': efficiency_before - efficiency_after,
            'flow_disruption': flow_disruption_value
        })
    
    return impacts

# 计算加权中心性
weighted_degree, weighted_betweenness, weighted_closeness = calculate_weighted_centralities(G, flows_df)

# 打印加权度中心性的前10个节点
print("Top 10 stations by Weighted Degree Centrality:")
for node, centrality in sorted(weighted_degree.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"{node}: {centrality:.4f}")

# 评估移除影响
weighted_impacts = assess_weighted_removal_impact(G, weighted_degree, flows_df)

print("\nImpact of removing top 3 stations (Weighted Network):")
for impact in weighted_impacts:
    print(f"Node: {impact['node']}")
    print(f"  Efficiency change: {impact['efficiency_change']:.4f}")
    print(f"  Flow disruption: {impact['flow_disruption']:.4f}")

Top 10 stations by Weighted Degree Centrality:
Stratford: 1.0000
Bank and Monument: 0.9373
Liverpool Street: 0.8033
Waterloo: 0.7881
Canary Wharf: 0.6368
Victoria: 0.6140
London Bridge: 0.5425
King's Cross St. Pancras: 0.5347
Highbury & Islington: 0.4260
Canada Water: 0.4118

Impact of removing top 3 stations (Weighted Network):
Node: Stratford
  Efficiency change: 1075.0999
  Flow disruption: 0.0747
Node: Bank and Monument
  Efficiency change: 4840.3601
  Flow disruption: 0.0700
Node: Liverpool Street
  Efficiency change: 1021.7888
  Flow disruption: 0.0600
